### Facial Recognition Assignment
     Simran Jot Kaur : 101803100
     Pranshu Goyal   : 101803102
     Vinay Singh     : 101803108
### COE 6 

In [1]:
# importing libraries
# !pip install facenet_pytorch
# !pip install opencv-python
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import time
import os

In [11]:
# initializing MTCNN and InceptionResnetV1 

mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40) # keep_all=False
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40) # keep_all=True
resnet = InceptionResnetV1(pretrained='vggface2').eval() 

In [16]:
# Read data from folder

dataset = datasets.ImageFolder('photos') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn0(img, return_prob=True) 
    if face is not None and prob>0.92:
        emb = resnet(face.unsqueeze(0)) 
        embedding_list.append(emb.detach()) 
        name_list.append(idx_to_class[idx])        

# save data
data = [embedding_list, name_list] 
torch.save(data, 'data.pt') # saving data.pt file

In [17]:
name_list


['pranshu',
 'pranshu',
 'pranshu',
 'pranshu',
 'pranshu',
 'simran',
 'simran',
 'simran',
 'simran',
 'simran',
 'vinay',
 'vinay',
 'vinay',
 'vinay',
 'vinay',
 'vinay']

In [9]:
data

[[tensor([[ 3.8779e-02,  4.0142e-02, -4.0917e-02,  3.3884e-02,  4.1319e-03,
           -8.7995e-03,  3.0617e-03, -2.4013e-04, -1.4984e-02, -8.0451e-03,
           -5.2956e-02, -3.1550e-02,  5.1956e-03, -2.8937e-02, -4.7702e-03,
           -1.6886e-03, -3.1951e-02,  6.2584e-02, -1.9412e-02,  5.6833e-02,
            4.0563e-02, -3.3239e-03, -2.0325e-02,  1.6025e-02,  6.2576e-02,
            5.7999e-02, -1.8648e-02, -2.9149e-02,  1.9190e-02, -3.8559e-02,
            9.8907e-03,  2.5949e-02, -8.2395e-02, -9.3074e-03,  2.1239e-02,
            3.9835e-02,  6.0713e-02, -5.2334e-02, -3.3129e-02,  4.2720e-02,
            2.0435e-02,  6.9290e-02, -5.8547e-02, -1.4986e-02, -2.5234e-02,
            1.9183e-02, -2.5937e-02,  4.4216e-02, -2.7338e-02, -3.7675e-02,
            2.0579e-02, -8.1553e-02, -6.9859e-02, -3.5040e-02, -4.1106e-02,
            2.5891e-02, -4.1312e-02,  6.5674e-03,  3.7710e-02, -4.9533e-02,
            3.1094e-02,  8.2079e-02, -5.6864e-02, -6.0476e-02, -5.6775e-02,
            

In [18]:
# Using webcam recognize face

# loading data.pt file
load_data = torch.load('data.pt') 
embedding_list = load_data[0] 
name_list = load_data[1] 

cam = cv2.VideoCapture(0) 

while True:
    ret, frame = cam.read()
    if not ret:
        print("fail to grab frame, try again")
        break
        
    img = Image.fromarray(frame)
    img_cropped_list, prob_list = mtcnn(img, return_prob=True) 
    
    if img_cropped_list is not None:
        boxes, _ = mtcnn.detect(img)
                
        for i, prob in enumerate(prob_list):
            if prob>0.70:
                emb = resnet(img_cropped_list[i].unsqueeze(0)).detach() 
                
                dist_list = [] # list of matched distances, minimum distance is used to identify the person
                
                for idx, emb_db in enumerate(embedding_list):
                    dist = torch.dist(emb, emb_db).item()
                    dist_list.append(dist)

                min_dist = min(dist_list) # get minumum dist value
                min_dist_idx = dist_list.index(min_dist) # get minumum dist index
                name = name_list[min_dist_idx] # get name corrosponding to minimum dist
                
                box = boxes[i] 
                
                original_frame = frame.copy() # storing copy of frame before drawing on it
                
                if min_dist<0.90:
                    frame = cv2.putText(frame, name+' '+str(min_dist), (box[0],box[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),1, cv2.LINE_AA)
                
                frame = cv2.rectangle(frame, (box[0],box[1]) , (box[2],box[3]), (255,0,0), 2)

    cv2.imshow("IMG", frame)
        
    
    k = cv2.waitKey(1)
    if k%256==27: # ESC
        print('Esc pressed, closing...')
        break
        
    elif k%256==32: # space to save image
        print('Enter your name :')
        name = input()
        
        # create directory if not exists
        if not os.path.exists('photos/'+name):
            os.mkdir('photos/'+name)
            
        img_name = "photos/{}/{}.jpg".format(name, int(time.time()))
        cv2.imwrite(img_name, original_frame)
        print(" saved: {}".format(img_name))
        
        
cam.release()
cv2.destroyAllWindows()
    

<ipython-input-18-11d3d31c18a3>:41: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame = cv2.putText(frame, name+' '+str(min_dist), (box[0],box[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),1, cv2.LINE_AA)
<ipython-input-18-11d3d31c18a3>:43: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame = cv2.rectangle(frame, (box[0],box[1]) , (box[2],box[3]), (255,0,0), 2)


Esc pressed, closing...
